In [3]:
import tensorflow as tf
import pandas as pd

# 열 이름
column_names = ["A", "P", "C", "LK", "WK", "A_Coef", "LKG", "target"]

# 파싱 함수
def parse_line_to_list(line):
    line = tf.strings.strip(line)
    parts = tf.strings.split(line, sep="\t")
    parts = tf.strings.to_number(parts, out_type=tf.float32)
    return parts

# 필터: 유효한 줄만 통과
def is_valid_line(line):
    line = tf.strings.strip(line)
    parts = tf.strings.split(line, sep="\t")
    return tf.equal(tf.size(parts), 8)

# Dataset 로딩
dataset = tf.data.TextLineDataset("./seeds_dataset.txt") \
            .filter(is_valid_line) \
            .map(parse_line_to_list)

# 리스트로 변환
data_list = [list(row.numpy()) for row in dataset]

# DataFrame으로 변환
df = pd.DataFrame(data_list, columns=column_names)

df

,A,P,C,LK,WK,A_Coef,LKG,target
0,15.260000,14.84,0.8710,5.763,3.312,2.221,5.220,1.0
1,14.880000,14.57,0.8811,5.554,3.333,1.018,4.956,1.0
2,14.290000,14.09,0.9050,5.291,3.337,2.699,4.825,1.0
3,13.840000,13.94,0.8955,5.324,3.379,2.259,4.805,1.0
4,16.139999,14.99,0.9034,5.658,3.562,1.355,5.175,1.0
...,...,...,...,...,...,...,...,...
194,12.190000,13.20,0.8783,5.137,2.981,3.631,4.870,3.0
195,11.230000,12.88,0.8511,5.140,2.795,4.325,5.003,3.0
196,13.200000,13.66,0.8883,5.236,3.232,8.315,5.056,3.0
197,11.840000,13.21,0.8521,5.175,2.836,3.598,5.044,3.0


In [4]:
print(df.isnull().sum())  

A         0
P         0
C         0
LK        0
WK        0
A_Coef    0
LKG       0
target    0
dtype: int64


In [5]:
X = df.drop('target',axis=1)
X.head()

,A,P,C,LK,WK,A_Coef,LKG
0,15.260000,14.84,0.8710,5.763,3.312,2.221,5.220
1,14.880000,14.57,0.8811,5.554,3.333,1.018,4.956
2,14.290000,14.09,0.9050,5.291,3.337,2.699,4.825
3,13.840000,13.94,0.8955,5.324,3.379,2.259,4.805
4,16.139999,14.99,0.9034,5.658,3.562,1.355,5.175


In [6]:
y = df["target"].astype(int) - 1

print(y.value_counts())

target
1    68
0    66
2    65
Name: count, dtype: int64


In [7]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test=train_test_split(X,
                                                  y,
                                                  test_size=0.2,
                                                  shuffle=True,
                                                  random_state=12)
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(159, 7) (159,)
(40, 7) (40,)


In [8]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Dense(64, activation='relu', input_shape=(7,)))
model.add(tf.keras.layers.Dense(64, activation='relu'))
model.add(tf.keras.layers.Dense(32, activation='relu'))
model.add(tf.keras.layers.Dense(16, activation='relu'))
model.add(tf.keras.layers.Dense(8, activation='sigmoid'))
model.add(tf.keras.layers.Dense(3, activation='softmax'))


# 모델 요약 보기
model.summary()

c:\Users\syjwk\anaconda3\envs\tf310\lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 64)             │           512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 64)             │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 16)             │           528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 8)              │           136 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 3)              │            27 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 7,443 (29.07 KB)

 Trainable params: 7,443 (29.07 KB)

 Non-trainable params: 0 (0.00 B)

In [11]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy

model.compile(
    optimizer=Adam(),
    loss=SparseCategoricalCrossentropy(),
    metrics=['accuracy']
)

# 학습 (y_train은 반드시 0부터 시작하는 정수여야 함!)
history = model.fit(X_train, y_train, epochs=30, batch_size=8, validation_split=0.1)


Epoch 1/30
18/18 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.8989 - loss: 0.3875 - val_accuracy: 0.9375 - val_loss: 0.3848
Epoch 2/30
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.9158 - loss: 0.3749 - val_accuracy: 0.9375 - val_loss: 0.3663
Epoch 3/30
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8873 - loss: 0.4016 - val_accuracy: 0.9375 - val_loss: 0.3650
Epoch 4/30
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.9013 - loss: 0.3976 - val_accuracy: 0.8125 - val_loss: 0.4748
Epoch 5/30
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8842 - loss: 0.3800 - val_accuracy: 0.9375 - val_loss: 0.3856
Epoch 6/30
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.9381 - loss: 0.3252 - val_accuracy: 0.9375 - val_loss: 0.3380
Epoch 7/30
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8745 - loss: 0.3860 - val_accuracy: 0.9375 - val_loss: 0.3528
Epoch 8/30
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8928 - loss: 0.3486 - val_accuracy: 0.9375 - val_loss

In [20]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense

inputs = Input(shape=(7,))
x = Dense(64, activation='relu')(inputs)
x = Dense(64, activation='relu')(x)
x = Dense(32, activation='relu')(x)
x = Dense(16, activation='relu')(x)
x = Dense(8, activation='sigmoid')(x)
prediction = Dense(3, activation='softmax')(x)

model = Model(inputs=inputs, outputs=prediction)

model.summary()


Model: "functional_7"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_2 (InputLayer)      │ (None, 7)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_12 (Dense)                │ (None, 64)             │           512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_13 (Dense)                │ (None, 64)             │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_14 (Dense)                │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_15 (Dense)                │ (None, 16)             │           528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_16 (Dense)                │ (None, 8)              │           136 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_17 (Dense)                │ (None, 3)              │            27 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 7,443 (29.07 KB)

 Trainable params: 7,443 (29.07 KB)

 Non-trainable params: 0 (0.00 B)

In [23]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy

model.compile(
    optimizer=Adam(),
    loss=SparseCategoricalCrossentropy(),
    metrics=['accuracy']
)

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, stratify=y)

history = model.fit(
    X_train, y_train,
    epochs=30,
    batch_size=16,
    validation_data=(X_val, y_val)
)


Epoch 1/30
10/10 ━━━━━━━━━━━━━━━━━━━━ 2s 26ms/step - accuracy: 0.9454 - loss: 0.3679 - val_accuracy: 0.8250 - val_loss: 0.4971
Epoch 2/30
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.9000 - loss: 0.4111 - val_accuracy: 0.8750 - val_loss: 0.4327
Epoch 3/30
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.9433 - loss: 0.3482 - val_accuracy: 0.8500 - val_loss: 0.4584
Epoch 4/30
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.9458 - loss: 0.3546 - val_accuracy: 0.9000 - val_loss: 0.4234
Epoch 5/30
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.9076 - loss: 0.3763 - val_accuracy: 0.8500 - val_loss: 0.4122
Epoch 6/30
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.9060 - loss: 0.3585 - val_accuracy: 0.7000 - val_loss: 0.5874
Epoch 7/30
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.7883 - loss: 0.4790 - val_accuracy: 0.8750 - val_loss: 0.4240
Epoch 8/30
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.8859 - loss: 0.4060 - val_accuracy: 0.8750 - va

In [27]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense

class SimpleMLP(Model):
    
    def __init__(self, num_classes):
        super(SimpleMLP, self).__init__(name='mlp')
        self.num_classes = num_classes
        
        self.dense1 = Dense(64, activation='relu')
        self.dense2 = Dense(64, activation='relu')
        self.dense3 = Dense(32, activation='relu')
        self.dense4 = Dense(16, activation='relu')
        self.dense5 = Dense(8, activation='sigmoid')
        self.dense6 = Dense(num_classes, activation='softmax')  
    
    def call(self, inputs):
        x = self.dense1(inputs)
        x = self.dense2(x)
        x = self.dense3(x)
        x = self.dense4(x)
        x = self.dense5(x)
        return self.dense6(x)


In [28]:
model = SimpleMLP(num_classes=3)
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

history = model.fit(X_train, y_train, epochs=30, batch_size=16, validation_split=0.2)

Epoch 1/30
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 25ms/step - accuracy: 0.3279 - loss: 1.2468 - val_accuracy: 0.1875 - val_loss: 1.1810
Epoch 2/30
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.3736 - loss: 1.0762 - val_accuracy: 0.1875 - val_loss: 1.1185
Epoch 3/30
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.3970 - loss: 1.0464 - val_accuracy: 0.1875 - val_loss: 1.0731
Epoch 4/30
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.3607 - loss: 1.0373 - val_accuracy: 0.1875 - val_loss: 1.0345
Epoch 5/30
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.3607 - loss: 1.0009 - val_accuracy: 0.2188 - val_loss: 0.9955
Epoch 6/30
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.3937 - loss: 0.9852 - val_accuracy: 0.4062 - val_loss: 0.9713
Epoch 7/30
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.4702 - loss: 0.9491 - val_accuracy: 0.4062 - val_loss: 0.9542
Epoch 8/30
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.5759 - loss: 0.9468 - val_accuracy: 0.5312 - val_loss: 0.9157
E